<a href="https://colab.research.google.com/github/Rogerio-mack/IMT_Banco_de_Dados/blob/main/valida_SQL_exercicios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [125]:
import os
import zipfile
import pandas as pd
import json
import re

import unicodedata

def remover_acentos(texto):
    nfkd_form = unicodedata.normalize('NFKD', texto)
    return "".join([c for c in nfkd_form if not unicodedata.combining(c)])

# Texto de validação
validacao = '''
-- EXERCÍCIO 1
FROM funcionarios
JOIN funcionarios
-- EXERCÍCIO 1b
GROUP BY
HAVING count(*) > 1;
-- EXERCÍCIO 2
LEFT JOIN
-- EXERCÍCIO 3
-- EXERCÍCIO 4
GROUP BY p.id, p.nome
-- EXERCÍCIO 5
EXISTS
-- EXERCÍCIO 6
NOT EXISTS
'''

def padronizar(texto):
    # Remove espaços extras e deixa minúsculo
    return re.sub(r'\s+', ' ', remover_acentos(texto.strip().lower()))

# validacao_std = padronizar(validacao)
validacao_std = remover_acentos(validacao.lower())

# Lista para os resultados finais
resultados = []

# Caminho da pasta
pasta = '/content'

# Percorre todos os arquivos zip
for nome_arquivo in os.listdir(pasta):
    if nome_arquivo.endswith('.zip'):
        caminho_zip = os.path.join(pasta, nome_arquivo)
        with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
            tmp_dir = os.path.join(pasta, 'temp_extract')
            if os.path.exists(tmp_dir):
                # Remove possível conteúdo anterior
                import shutil
                shutil.rmtree(tmp_dir)
            os.makedirs(tmp_dir, exist_ok=True)
            zip_ref.extractall(tmp_dir)

        # Para cada arquivo extraído
        for arquivo in os.listdir(tmp_dir):
            caminho_arquivo = os.path.join(tmp_dir, arquivo)
            try:
                with open(caminho_arquivo, 'r', encoding='utf-8') as f:
                    conteudo = f.read()
            except:
                continue

            extensao = os.path.splitext(arquivo)[1].lower()

            # Se for notebook, tentar extrair código
            if extensao == '.ipynb':
                try:
                    nb = json.loads(conteudo)
                    code_cells = []
                    for cell in nb.get('cells', []):
                        if cell.get('cell_type') == 'code':
                            code_cells.extend(cell.get('source', []))
                    conteudo_code = '\n'.join(code_cells)
                    conteudo_std = padronizar(conteudo_code)
                except:
                    continue
            elif extensao == '.sql':
                conteudo_std = padronizar(conteudo)
            else:
                # Ignorar outros tipos
                continue

            # print(conteudo_std)
            # break

            linhas = conteudo_std.splitlines()
            total_linhas = len(linhas)

            # Contar linhas de comentários e comandos
            linhas_comentario = sum(1 for l in validacao_std.splitlines() if l.strip().startswith('--'))
            linhas_comando = len(validacao_std.splitlines()) - linhas_comentario

            # Verifica se cada linha de validação está presente no conteúdo
            presentes_completo = True
            comentarios_faltando = 0
            comandos_faltando = 0
            for linha_valid in validacao_std.splitlines():
                if linha_valid.strip().startswith('--'):
                    # print(linha_valid)
                    # Comentário: verificar presença
                    if linha_valid.strip() in conteudo_std:
                        comentarios_faltando += 1
                        # print('comentarios',comentarios_faltando)
                        presentes_completo = False

            for linha_valid in validacao_std.splitlines():
                if not linha_valid.strip().startswith('--'):
                    # print(linha_valid)
                    # Conteúdo: verificar presença
                    if linha_valid.strip() in conteudo_std:
                        comandos_faltando += 1
                        # print('comandos',comandos_faltando)
                        presentes_completo = False

            # Calcular porcentagens
            comentarios_perc = (comentarios_faltando / linhas_comentario) * 100 if linhas_comentario > 0 else 0
            comandos_perc = (comandos_faltando / linhas_comando) * 100 if linhas_comando > 0 else 0

            # print(comentarios_perc, comentarios_faltando, linhas_comentario)
            # print(comandos_perc, comandos_faltando, linhas_comando)

            # Obter o nome do arquivo até o 3o '_' ou o que vier primeiro
            nome_base = os.path.splitext(arquivo)[0]
            match_nome = re.match(r'^([^_]+)', nome_base)
            arquivo_substring = match_nome.group(1) if match_nome else nome_base

            # Montar o dicionário de cada arquivo
            resultados.append({
                'arquivo': arquivo_substring,
                'extensão': extensao,
                'comentarios': comentarios_faltando,
                'comentarios_perc': comentarios_perc,
                'comandos': comandos_faltando,
                'comandos_perc': comandos_perc
            })

        # Remove a pasta de extração após o processamento
        import shutil
        shutil.rmtree(tmp_dir)

# Criar DataFrame final
df_resultado = pd.DataFrame(resultados)

#
# df_resultado.sort_values('arquivo')
df_resultado.sort_values('comandos_perc')

,arquivo,extensão,comentarios,comentarios_perc,comandos,comandos_perc
6,passianigiuliazaparolli,.ipynb,0,0.0,4,44.444444
0,teradabrenogaravelo,.sql,7,100.0,5,55.555556
21,rosnerpedrohenrique,.sql,7,100.0,5,55.555556
16,passianigiuliazaparolli,.sql,7,100.0,5,55.555556
26,castellanimarianaboschetti,.sql,7,100.0,5,55.555556
31,scheidtfouadkhayat,.sql,7,100.0,5,55.555556
4,setteenzopizzonide,.sql,7,100.0,7,77.777778
3,baptistadanielshinjifunakipereira,.sql,7,100.0,7,77.777778
17,teinthalesxaviervon,.sql,7,100.0,7,77.777778
20,santanadanielcruz,.sql,7,100.0,7,77.777778


In [111]:
validacao_std.splitlines()[1].strip().startswith('--')

True

In [92]:
'join departamentos' in conteudo_std

True

In [105]:
# prompt: exclua da string validacao_std.splitlines()[2] brancos ao final

validacao_std.splitlines()[2].strip() in conteudo_std

True

In [81]:
            # Calcular porcentagens
            comentarios_perc = (comentarios_faltando / linhas_comentario) * 100 if linhas_comentario > 0 else 0
            comandos_perc = (comandos_faltando / linhas_comando) * 100 if linhas_comando > 0 else 0

90.9090909090909

In [ ]:
linha_valid.strip().startswith('--'):
                    # Comentário: verificar presença
                    if linha_valid.strip() not in conteudo_std:

In [85]:
            comentarios_faltando = 0
            comandos_faltando = 0
            for linha_valid in validacao_std.splitlines():
                print(linha_valid)
                if linha_valid.strip().startswith('--'):
                    # Comentário: verificar presença
                    if linha_valid.strip() not in conteudo_std:
                        comentarios_faltando += 1
                        print('comentarios',comentarios_faltando)
                        presentes_completo = False
                elif linha_valid.strip() != '':
                    # Conteúdo: verificar presença
                    if linha_valid.strip() not in conteudo_std:
                        comandos_faltando += 1
                        print('comandos',comandos_faltando)
                        presentes_completo = False


-- questao 1.a) 
comentarios 1
join departamentos 
comandos 1
order by d.nome
comandos 2
where f.id_departamento = d.id_departamento
comandos 3
order by d.nome;
comandos 4
-- questao 1.b) 
comentarios 2
where d.localizacao =  
comandos 5
-- questao 1.c) 
comentarios 3
left join departamentos 
comandos 6
right join funcionarios 
comandos 7
-- questao 1.d) 
comentarios 4
where f.nome like 
comandos 8
-- questao 1.e)
comentarios 5
join funcionarios 
comandos 9
select avg(salario) 
comandos 10


In [57]:
sum(1 for l in validacao_std.splitlines() if l.strip().startswith('--'))

5

In [56]:
for l in validacao_std.splitlines()

['',
 '-- questao 1.a) ',
 'join departamentos ',
 'order by d.nome',
 'where f.id_departamento = d.id_departamento',
 'order by d.nome;',
 '-- questao 1.b) ',
 'where d.localizacao =  ',
 '-- questao 1.c) ',
 'left join departamentos ',
 'right join funcionarios ',
 '-- questao 1.d) ',
 'where f.nome like ',
 '-- questao 1.e)',
 'join funcionarios ',
 'select avg(salario) ']

In [34]:
'-- questao 1.a)' in conteudo_std

True

In [39]:
'-- questao 1.a)' in remover_acentos(validacao_std.splitlines()[1].strip())

True

In [49]:
validacao_std.splitlines()[1].strip().startswith('--')

True

In [38]:
# prompt: crie uma função que exclui acentos em uma variável texto

import unicodedata

def remover_acentos(texto):
    nfkd_form = unicodedata.normalize('NFKD', texto)
    return "".join([c for c in nfkd_form if not unicodedata.combining(c)])

# Exemplo de uso:
# texto_com_acentos = "Esta é uma string com acentos, ç, ã, õ."
# texto_sem_acentos = remover_acentos(texto_com_acentos)
# print(texto_sem_acentos)


In [42]:
validacao_std.splitlines()[1].strip() in conteudo_std

True

In [19]:
# prompt: Crie um arquivo .txt com com o conteúdo da variável validacao = '''
# -- Questão 1.a)
# JOIN departamentos
# ORDER BY d.nome
# WHERE f.id_departamento = d.id_departamento
# ORDER BY d.nome;
# -- Questão 1.b)
# WHERE d.localizacao =
# -- Questão 1.c)
# LEFT JOIN departamentos
# RIGHT JOIN funcionarios
# -- Questão 1.d)
# WHERE f.nome LIKE
# -- Questão 1.e)
# JOIN funcionarios
# SELECT AVG(salario)
# ''' e um código para ler cada linha desse arquivo e imprimir.

# Cria o arquivo validacao.txt com o conteúdo da variável validacao
with open("validacao.txt", "w") as f:
  f.write(validacao.lower())

# Lê cada linha do arquivo validacao.txt e imprime
with open("validacao.txt", "r") as f:
  for linha in f:
    print(linha.strip()) # Use strip() para remover quebras de linha


-- questão 1.a)
join departamentos
order by d.nome
where f.id_departamento = d.id_departamento
order by d.nome;
-- questão 1.b)
where d.localizacao =
-- questão 1.c)
left join departamentos
right join funcionarios
-- questão 1.d)
where f.nome like
-- questão 1.e)
join funcionarios
select avg(salario)


In [12]:
validacao_std

'-- questão 1.a) join departamentos order by d.nome where f.id_departamento = d.id_departamento order by d.nome; -- questão 1.b) where d.localizacao = -- questão 1.c) left join departamentos right join funcionarios -- questão 1.d) where f.nome like -- questão 1.e) join funcionarios select avg(salario)'

In [9]:
linha_stripped

"-- questao 1.a) select f.nome, f.cargo, f.salario, d.nome as departamento from funcionarios f inner join departamentos d on f.id_departamento = d.id_departamento order by d.nome; select f.nome, f.cargo, f.salario, d.nome as departamento from funcionarios f inner join departamento d using (id_departamento) order by d.nome; select f.nome, f.cargo, f.salario, d.nome as departamento from funcionarios f where departamentos d on f.id_departamento = d.id_departamento order by d.nome; -- questao 1.b) select f.nome, f.cargo, f.salario, d.nome as departamento from funcionarios f inner join departamentos d on f.id_departamento = d.id_departamento where d.localizacao = 'predio a' order by d.nome; --questao 1.c) select f.nome, f.cargo, f.salario, d.nome as departamento from funcionarios f left join departamentos on f.id_departamento = d.id_departamento; select f.nome, f.cargo, f.salario, d.nome as departamento from funcionarios f right join departamentos on f.id_departamento = d.id_departamento; -